# load data

In [1]:
import json

In [5]:
prefix = './data_ch/'

train = []
files = ['train.0']
for file_item in files:
    with open(prefix+file_item, 'r') as f:
        for item in f:
            d = {}
            d['label'] = 0
            d['text'] = " ".join(jieba.cut(item.strip().replace("臺", "台"), cut_all=False))
            train.append(d)
files = ['train.1']
for file_item in files:
    with open(prefix+file_item, 'r') as f:
        for item in f:
            d = {}
            d['label'] = 1
            d['text'] = " ".join(jieba.cut(item.strip().replace("臺", "台"), cut_all=False))
            train.append(d)

valid = []
files = ['dev.0']
for file_item in files:
    with open(prefix+file_item, 'r') as f:
        for item in f:
            d = {}
            d['label'] = 0
            d['text'] = " ".join(jieba.cut(item.strip().replace("臺", "台"), cut_all=False))
            valid.append(d)
files = ['dev.1']
for file_item in files:
    with open(prefix+file_item, 'r') as f:
        for item in f:
            d = {}
            d['label'] = 1
            d['text'] = " ".join(jieba.cut(item.strip().replace("臺", "台"), cut_all=False))
            valid.append(d)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.648 seconds.
Prefix dict has been built successfully.


In [6]:
print(len(train), len(valid))

19906 2488


In [34]:
train[-31]

{'label': 1,
 'text': '然而 在 當前 的 政治 環境 下 ， 制憲權 的 行使 ， 卻 被 視為 破壞 憲政 秩序 的 行為 ， 無怪乎 國軍將 領會 拒斥 新憲法 的 產生 ， 惟 就 各國 憲政 發展 史觀 之'}

# build embedding

In [2]:
from multiprocessing import Pool
import pickle
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
SENT_DETECTOR = nltk.data.load('tokenizers/punkt/english.pickle')
import jieba
import jieba.analyse
import re 
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import os

[nltk_data] Downloading package punkt to /home/cealia312/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def get_tokens(s):
    return s.split()

def collect_words(data, n_workers): #collect training, valid words
    sent_list = []
    for i in data:
        sent_list.extend(i['text'].split('\n'))
        
    chunks = \
    [' '.join(sent_list[i: i+len(sent_list)//n_workers]) for i in range(0, len(sent_list), len(sent_list)//n_workers)]
    
    with Pool(n_workers) as pool:
        chunks = pool.map_async(get_tokens, chunks)
        words = set(sum(chunks.get(), []))
    
    return words

In [4]:
words = set()
print('Collect train words...')
words |= collect_words(train, n_workers=6)
print('Collect valid words...')
words |= collect_words(valid, n_workers=6)
# print('Collect test words...')
# words |= collect_words(test, n_workers=6)

Collect train words...


NameError: name 'train' is not defined

In [51]:
len(words)

46064

In [8]:
# lower_words = set()

# for word in words:
#     word = word.lower()
#     lower_words.add(word)

In [52]:
lower_words = words

In [10]:
# len(lower_words)

213549

In [53]:
for i in ['<pad>', '<unk>', '<sos>', '<eos>']:
    lower_words.add(i)

In [54]:
len(lower_words)

46068

In [13]:
# used_glove = {}
# from_glove = 0
# with open('glove.840B.300d.txt') as fp:
#     row1 = fp.readline()
#     # if the first row is not header
#     if not re.match('^[0-9]+ [0-9]+$', row1):
#         # seek to 0
#         fp.seek(0)
#     # otherwise ignore the header

#     for i, line in tqdm(enumerate(fp), total=2196017):
#         cols = line.rstrip().split(' ')
#         word = cols[0].lower()
        
#         if word in lower_words:
#             from_glove += 1
#             used_glove[word] = [float(v) for v in cols[1:]]

100%|██████████| 2196017/2196017 [00:46<00:00, 47564.10it/s]


In [15]:
# len(used_glove)

146865

In [60]:
used_glove = {}
# special_tokens = ['<pad>', '<unk>', '<sos>', '<eos>']
for t in lower_words:
#     if t not in used_glove:
        used_glove[t] = np.random.normal(scale=0.6, size=(256, ))

In [61]:
len(used_glove)

46068

In [62]:
weights_matrix = list(used_glove.values())

In [63]:
np.shape(weights_matrix)

(46068, 256)

In [64]:
weights_matrix = np.asarray(weights_matrix)

In [65]:
weights_matrix.shape

(46068, 256)

In [66]:
word2id = {}
id2word = {}
# words_found = 0

for i, word in tqdm(enumerate(used_glove.keys()), total=len(used_glove)):
    word2id[word] = i
    id2word[i] = word
#     try: 
#         weights_matrix[i] = used_glove[word]
#         words_found += 1
#     except KeyError:
#         weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

100%|██████████| 46068/46068 [00:00<00:00, 1404054.68it/s]


In [67]:
for i in ['<pad>', '<unk>', '<sos>', '<eos>']:
    print(word2id[i])

16183
44230
15134
16044


In [18]:
# weights_matrix.shape (213553, 300)

(213553, 300)

In [68]:
# import os
# import json
# prefix = 'word_dict_ch/'
# os.mkdir(prefix)
# with open(prefix+'word2id.json', 'w') as f:
#     json.dump(word2id, f)
    
# with open(prefix+'id2word.json', 'w') as f:
#     json.dump(id2word, f)
    
# with open(prefix+'weights_matrix.pkl','wb') as f:
#     pickle.dump(weights_matrix, f)

In [7]:
with open('word_dict_ch/word2id.json', 'r') as f:
    word2id = json.load(f)
    
with open('word_dict_ch/id2word.json', 'r') as f:
    id2word = json.load(f)
    
with open('word_dict_ch/weights_matrix.pkl','rb') as f:
    weights_matrix = pickle.load(f)

In [8]:
weights_matrix.shape

(46068, 256)

## create dataset

* preprocess

In [9]:
from concurrent.futures import ProcessPoolExecutor as P
from functools import partial

In [10]:
# prefix = './data/'
# train = []
# files = ['democratic_only.train.en', 'republican_only.train.en']
# for file_item in files:
#     with open(prefix+file_item, 'r') as f:
#         for item in f:
#             item = item.strip()
#             train.append(item)
# #             word_list = nltk.word_tokenize(item)
# #             for word in word_list:
# #                 word = word.lower()
# #                 if word not in word_to_id:
# #                     word_to_id[word] = 0
# #                 word_to_id[word] += 1
# prefix = './data/'
# valid = []
# files = ['democratic_only.dev.en', 'republican_only.dev.en']
# for file_item in files:
#     with open(prefix+file_item, 'r') as f:
#         for item in f:
#             item = item.strip()
#             valid.append(item)

In [11]:
train[:3]

[{'label': 0,
  'text': '9 年 ， 面對 複 雜嚴峻 的 台海 形勢 ， 我們 深入 貫徹習 近平 總書記 在 《 告台灣 同胞 書 》 發表 40 週年 紀念會 上 的 重要 講話 和 黨 的 十九 屆 四中 全會 精神'},
 {'label': 0,
  'text': '9 年 ， 廣大台灣 同胞 對 加強 兩岸 交流 合作 的 期待 與民 進黨 當局 的 限制 打壓 形成 了 強烈 對 比 ； 兩岸 人員往 來 、 各 領域 交流 合作 發展 的 勢頭 與 民進 黨 當局 的 倒行逆施 形成 了 強烈 反差'},
 {'label': 0,
  'text': '續 出台 關於進 一步 促進 兩岸 經濟 文化交流 合作 的 一系列 政策措施 ， 為 台灣 同胞 來大陸 學習 、 工作 、 生活 提供 更 多 的 便利 ， 積極 鼓勵 、 支援 兩岸民間 各 領域 的 交流 合作'}]

In [12]:
def chunk(data, n):
    for i in range(0, len(data), n):
        yield data[i:i+n]
        
def raw2processed(data):
    processed_data = []
    
    for d in data:
        item = {}
        #item['id'] = d['id']
        item['text'] = []
        item['label'] = d['label']
        for w in d['text'].split():
            #w = w.lower()
            if w in word2id:
                item['text'].append(word2id[w])
            else:
                item['text'].append(word2id['<unk>'])
                
        processed_data.append(item)
    return processed_data
                    
def preprocessed_ls(num_worker, raw_data):
    chunked_data = list(chunk(raw_data, 100))

    with P(max_workers=num_worker) as executor:
        r = executor.map(raw2processed, chunked_data)
    ls_r = list(r)
    return [item for ls in ls_r for item in ls]

In [13]:
print('Preprocess training corpus to indexes...')
train_ls = preprocessed_ls(num_worker=4, raw_data=train)
print('Preprocess valid corpus to indexes...')
valid_ls = preprocessed_ls(num_worker=4, raw_data=valid)
# print('Preprocess testing corpus to indexes...')
# test_ls = preprocessed_ls(num_worker=8, raw_data=test)

Preprocess training corpus to indexes...
Preprocess valid corpus to indexes...


In [14]:
print(train[0], train_ls[0])
print(train[1], [id2word[str(id_)] for id_ in train_ls[1]['text']])

{'label': 0, 'text': '9 年 ， 面對 複 雜嚴峻 的 台海 形勢 ， 我們 深入 貫徹習 近平 總書記 在 《 告台灣 同胞 書 》 發表 40 週年 紀念會 上 的 重要 講話 和 黨 的 十九 屆 四中 全會 精神'} {'text': [21182, 10148, 7519, 12807, 23901, 520, 28393, 45498, 38522, 7519, 5140, 6745, 428, 5283, 31723, 3017, 18306, 32870, 7405, 12688, 11309, 13839, 28678, 40429, 18290, 1802, 28393, 12113, 16362, 23066, 24107, 28393, 19082, 16135, 36019, 1568, 39836], 'label': 0}
{'label': 0, 'text': '9 年 ， 廣大台灣 同胞 對 加強 兩岸 交流 合作 的 期待 與民 進黨 當局 的 限制 打壓 形成 了 強烈 對 比 ； 兩岸 人員往 來 、 各 領域 交流 合作 發展 的 勢頭 與 民進 黨 當局 的 倒行逆施 形成 了 強烈 反差'} ['9', '年', '，', '廣大台灣', '同胞', '對', '加強', '兩岸', '交流', '合作', '的', '期待', '與民', '進黨', '當局', '的', '限制', '打壓', '形成', '了', '強烈', '對', '比', '；', '兩岸', '人員往', '來', '、', '各', '領域', '交流', '合作', '發展', '的', '勢頭', '與', '民進', '黨', '當局', '的', '倒行逆施', '形成', '了', '強烈', '反差']


In [15]:
print(len(train_ls), len(valid_ls))

19906 2488


* torch Dataset

In [16]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from functools import partial

In [17]:
class Article_Dataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __getitem__(self, index):#每次怎麼讀數據
        return self.data[index]
    
    def __len__(self):
        return len(self.data)

In [18]:
def collate_fn(datas): #整理func -> batch = [(),(),]
    batch_text = []
#     batch_id = []
#     batch_summary = []
    batch_label = []

    for data in datas:
        text_max_len=80
        pad_text = []
        if len(data['text']) >= text_max_len-2:
            pad_text.append([word2id['<sos>']] + data['text'][:text_max_len-2] + [word2id['<eos>']])
        else:
            pad_text.append\
            ([word2id['<sos>']] + data['text'][:len(data['text'])] + \
             [word2id['<eos>']] + [word2id['<pad>']]*(text_max_len-len(data['text'])-2))
        batch_text.extend(pad_text)
        #label
        batch_label.append(data['label'])
#         #summary=30
#         if 'summary' in data:
#             pad_sum = []
#             if len(data['summary']) >= 28:
#                 pad_sum.append([word2id['<sos>']] + data['summary'][:30-2] + [word2id['<eos>']])
#             else:
#                 pad_sum.append\
#                 ([word2id['<sos>']] + data['summary'] + \
#                  [word2id['<eos>']] + [word2id['<pad>']]*(30-len(data['summary'])-2))
#             batch_summary.extend(pad_sum)
#         else:
#             batch_summary.append([word2id['<sos>']]*30)
    return torch.LongTensor(batch_text), torch.tensor(batch_label)

In [19]:
trainset = Article_Dataset(train_ls)
validset = Article_Dataset(valid_ls)
# testset = Article_Dataset(test_ls)

In [20]:
# next(iter(trainset))

In [21]:
batched_trainset = DataLoader(trainset, batch_size=2, shuffle=True, num_workers=0, \
                            collate_fn=lambda x:collate_fn(x))
# batched_testset = DataLoader(testset, batch_size=2, shuffle=False, num_workers=0, \
#                             collate_fn=lambda x:collate_fn(x))

In [22]:
next(iter(batched_trainset))

(tensor([[15134, 34276, 13867, 18556, 28393, 43125, 15923, 28393, 27352, 42247,
           5051,  9473, 44624, 28393, 19947,  7519, 31902, 35328,  7519, 43125,
          15923,  6174, 32728, 13381,  7519,  3017, 32557, 10837, 40456,  7519,
          29981, 37874, 19532, 26364, 13565, 36421, 16146, 18748,  7363, 28393,
          16044, 16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183,
          16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183,
          16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183,
          16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183, 16183],
         [15134,  6382,  3017, 26164, 36530, 31250, 33824, 20934, 35438, 38249,
           7519,   892, 16146, 33824, 28083, 18723, 38249, 28393, 25877, 25277,
           7519, 36765, 45837, 42770, 17466, 28393,  9558, 22009,  2684, 26164,
          23119,  6833, 28393, 17059, 21388, 16044, 16183, 16183, 16183, 16183,
          16183, 16183, 16183, 16183, 1

In [23]:
# next(iter(batched_testset))

# Model

In [24]:
import torch.nn as nn
import torch
import random
import torch.optim as optim

In [25]:
print(torch.__version__)
print(torch.cuda.is_available(), torch.backends.cudnn.enabled)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
print(device)

1.5.1
True True
cuda


In [26]:
class Seq2Seq(nn.Module):
    '''
    1. get word embedding 
    2. encaptulate Encoder and Decoder
    '''
    def __init__(self, weight_matrix, encoder, decoder):
        super().__init__()
        word_num, input_size = weight_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weight_matrix))
        self.embedding.weight.requires_grad = True
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, input_indexes, output_indexes, teacher_forcing):
        batch_size, output_seq_len = output_indexes.shape
        
        input_ = self.embedding(input_indexes) #input_=(batch_size, input_len, input_size)
        pre_hid = self.encoder(input_) #pre_hid=(num_layers * num_directions, batch_size, hidden_size)
        latent = pre_hid.clone().detach().squeeze() #latent=(batch_size, hidden_size)
        decoder_input = self.embedding(output_indexes[:,0])
        decoder_input = decoder_input.unsqueeze(1) #input_len=1
        
        output = torch.zeros(output_seq_len, batch_size, self.embedding.num_embeddings).to(device)
        output_2 = torch.zeros(output_seq_len, batch_size).to(device)
        for i in range(1, output_seq_len):
            word_dist, pre_hid = self.decoder(decoder_input, pre_hid)
            word_indexes = torch.argmax(word_dist, dim=1)
            output[i] = word_dist
            output_2[i] = word_indexes
            
            if random.random()>=teacher_forcing: #if teacher_forcing==0, turn off 
                decoder_input = self.embedding(word_indexes)
            else:
                decoder_input = self.embedding(output_indexes[:, i])
            decoder_input = decoder_input.unsqueeze(1)
        
        output_2 = torch.transpose(output_2, 0, 1)
        return output, output_2, latent
    
    def encode(self, input_indexes):
        input_ = self.embedding(input_indexes) #input_=(batch_size, input_len, input_size)
        pre_hid = self.encoder(input_) #pre_hid=(num_layers * num_directions, batch_size, hidden_size)
        latent = pre_hid.clone().detach().squeeze() #latent=(batch_size, hidden_size)
        return latent
    
    def decode(self, output_indexes, latent): #teacher_forcing=0
        pre_hid = latent.unsqueeze(0)
        batch_size, output_seq_len = output_indexes.shape
        
        decoder_input = self.embedding(output_indexes[:,0])
        decoder_input = decoder_input.unsqueeze(1) #input_len=1
        
        output = torch.zeros(output_seq_len, batch_size, self.embedding.num_embeddings).to(device)
        output_2 = torch.zeros(output_seq_len, batch_size).to(device)
        for i in range(1, output_seq_len):
            word_dist, pre_hid = self.decoder(decoder_input, pre_hid)
            word_indexes = torch.argmax(word_dist, dim=1)
            output[i] = word_dist
            output_2[i] = word_indexes
            
            decoder_input = self.embedding(output_indexes[:, i])
            decoder_input = decoder_input.unsqueeze(1)
            
        output_2 = torch.transpose(output_2, 0, 1)
        return output, output_2

In [27]:
class Encoder(nn.Module):
    """
    encode the whole input sequence
    """
    def __init__(self, input_size, hid_size, num_layers, bidirectional):
        super().__init__()
        self.rnn = nn.GRU(input_size, hid_size, num_layers, batch_first=True, bidirectional=bool(bidirectional))
        
    def forward(self, input_):
        output, h_n = self.rnn(input_) 
        #input_=(batch_size, seq_len, feature) 
        #h_0=(num_layers * num_directions, batch_size, hidden_size)
        
        #output=(batch_size, seq_len, num_directions * hidden_size)
        #h_n=(num_layers * num_directions, batch_size, hidden_size)
        
        return h_n

In [28]:
class Decoder(nn.Module):
    """
    dose a single step of decoding
    """
    def __init__(self, input_size, hid_size, num_layers, bidirectional, word_num):
        super().__init__()
        self.rnn = nn.GRU(input_size, hid_size, num_layers, batch_first=True, bidirectional=bool(bidirectional))
        self.lr = nn.Linear((bidirectional+1)*hid_size, word_num)
    
    def forward(self, input_, pre_hid):
        output, h_n = self.rnn(input_, pre_hid)
        #input_=(batch_size, seq_len, feature) 
        #h_0=(num_layers * num_directions, batch_size, hidden_size)
        
        #output=(batch_size, seq_len, num_directions * hidden_size)
        #h_n=(num_layers * num_directions, batch_size, hidden_size)
        
        word_dist = self.lr(output.squeeze(1))
        return word_dist, h_n

In [29]:
class Classifier(nn.Module):
    def __init__(self, latent_size, class_num):
        super().__init__()
        self.fc1 = nn.Linear(latent_size, 100)
        self.relu1 = nn.LeakyReLU(0.3)
        self.fc2 = nn.Linear(100, 50)
        self.relu2 = nn.LeakyReLU(0.2)
        self.fc3 = nn.Linear(50, class_num)

    def forward(self, input): #input=(batch_size, hidden_size)
        out = self.fc1(input)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out  # batch_size * label_size

In [30]:
# x = torch.randn(2, 2, requires_grad=True)
# y = x.clone().detach()
# # y.retain_grad()
# # y.requires_grad = True

# z = y**2
# z.mean().backward()

# print(y.grad)
# print(x.grad)

In [31]:
word_num, emb_dim = weights_matrix.shape
encoder = Encoder(input_size=256, hid_size=256, num_layers=1, bidirectional=0)
decoder = Decoder(input_size=256, hid_size=256, num_layers=1, bidirectional=0, word_num=word_num)
model = Seq2Seq(weight_matrix=weights_matrix, encoder=encoder, decoder=decoder)
classifier = Classifier(latent_size=256, class_num=2)

In [32]:
model.load_state_dict(torch.load('model_ch/model_1.pt'))
classifier.load_state_dict(torch.load('model_ch/cls_1.pt'))

<All keys matched successfully>

In [33]:
model.to(device)
classifier.to(device)
print(model)
print(classifier)

Seq2Seq(
  (embedding): Embedding(46068, 256)
  (encoder): Encoder(
    (rnn): GRU(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (rnn): GRU(256, 256, batch_first=True)
    (lr): Linear(in_features=256, out_features=46068, bias=True)
  )
)
Classifier(
  (fc1): Linear(in_features=256, out_features=100, bias=True)
  (relu1): LeakyReLU(negative_slope=0.2)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (relu2): LeakyReLU(negative_slope=0.2)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)


In [34]:
# model.embedding.weight.requires_grad = False

In [35]:
for name, module in model.named_parameters():
    print(name, module)

embedding.weight Parameter containing:
tensor([[ 0.2723,  0.0410, -0.4631,  ..., -0.0617, -0.0115,  0.1427],
        [-1.1988,  0.1730,  0.5648,  ..., -0.1958, -0.6262,  0.3177],
        [-0.4548, -0.0784, -0.3080,  ...,  0.0450,  0.7476,  0.8364],
        ...,
        [-0.5624, -0.5748,  0.7006,  ..., -0.1694,  0.4826,  0.0705],
        [ 0.3107, -0.1274, -0.1226,  ..., -0.3409,  0.5496, -0.4199],
        [-0.2818, -1.4297, -0.3846,  ..., -0.7473, -0.4242,  0.5158]],
       device='cuda:0', requires_grad=True)
encoder.rnn.weight_ih_l0 Parameter containing:
tensor([[-0.0038,  0.0264, -0.1779,  ..., -0.0441, -0.0586, -0.0453],
        [ 0.0290, -0.0666, -0.1162,  ...,  0.0242,  0.0197,  0.0008],
        [ 0.1801, -0.0587, -0.0035,  ...,  0.2067, -0.1527, -0.0127],
        ...,
        [-0.0162,  0.0944, -0.0278,  ...,  0.0022,  0.0382, -0.0462],
        [-0.0375, -0.2420,  0.0680,  ..., -0.0404,  0.0764, -0.1142],
        [ 0.0688,  0.0933, -0.0431,  ...,  0.0934, -0.0938,  0.0586]],
  

In [36]:
print(model.embedding.weight.requires_grad)

True


In [35]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=word2id['<pad>'])
cls_optimizer = optim.Adam(classifier.parameters(), lr=1e-4)
cls_criterion = nn.CrossEntropyLoss()

In [36]:
# device='cpu'

In [37]:
def run_epoch(mode, batch_size):
    #prepare dataloader
    if mode=='train':
        dataset = trainset
        shuffle = True
        teacher_forcing = 1
        model.train()
    elif mode=='valid':
        dataset = trainset
        shuffle = False
        teacher_forcing = 0
        model.eval() #set the module in evaluation mode, only affect dropout, batchnorm
    elif mode=='test':
        dataset = testset
        shuffle = False
        teacher_forcing = 0
        model.eval()
    
    batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4, \
                            collate_fn=lambda x:collate_fn(x))
    #train model
    t = tqdm(enumerate(batched_dataset), total=len(dataset)/batch_size)
    total_loss = 0
    cls_total_loss = 0
    ans = []
    for i, batch in t: #run batch
        batch_text = batch[0].to(device)
        batch_label = batch[1].to(device)
        output, output_indexes, latent = model(batch_text, batch_text, teacher_forcing=teacher_forcing) 
        #output=(output_seq_len, batch_size, word num) #latent=(batch_size, hidden_size), requires_grad=True
        cls_output = classifier(latent) #cls_output=(batch_size, label_size=2)
#         print(latent.shape)
#         print(latent.requires_grad)
#         print(cls_output.shape, batch_label.shape)
        
        output_seq_len, batch_size, word_num = output.shape
        output = output[1:].view(-1, word_num)
        batch_size, output_seq_len = batch_text.shape
        batch_text_trans = torch.zeros(output_seq_len, batch_size, dtype=torch.long).to(device)
        for seq_p in range(output_seq_len):
            batch_text_trans[seq_p] = batch_text[:, seq_p]
        batch_text_trans = batch_text_trans[1:]
        batch_text_trans = batch_text_trans.view(-1)
        
        loss = criterion(output, batch_text_trans)
#         loss.grad_fn
        total_loss += loss.item()
        cls_loss = cls_criterion(cls_output, batch_label)
        cls_total_loss += cls_loss.item()
    
        if mode=='train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step() #based on the (1)current gradient (2)the update rule
            cls_optimizer.zero_grad()
            cls_loss.backward()
            cls_optimizer.step() #based on the (1)current gradient (2)the update rule
            
        t.set_postfix({'ae loss':total_loss/(i+1), 'cls loss':cls_total_loss/(i+1)})
        
        for indexes in output_indexes:
#             indexes = indexes.type(torch.uint8)
            ans.append(indexes.tolist())
    return ans

In [81]:
ans = run_epoch('train', 32)

623it [07:33,  1.37it/s, ae loss=2.7, cls loss=0.093]                                


In [83]:
for indexes in list(ans[:50]):
    sen = ""
    for w_id in indexes[1:]:
        sen += id2word[str(int(w_id))]#+" "
    print(sen)

體處理業務範圍，我這裡各位我會公佈，請發言人等待<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>
十二月十二月七日，在日本的記者獨聯盟召開記者記者會指出指出日本先生指出，與中村輝夫的，肉體，，休養，表示台表示還台灣<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>
道，卡式有有卡式台胞證以來，有關部門依法依法台灣同胞，了萬餘本萬餘本，依法依法保護、便捷、的有了台胞的好事<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>
如果如果大家不知道的，，如果如果的說法目還，陳的的但是的國民黨的話，也也的話也是黨念茲來，，黨念茲的立法院，「不」話」，？<eos>說「<eos

In [38]:
ans_valid = run_epoch('valid', 32)

623it [01:38,  6.32it/s, ae loss=6.27, cls loss=0.0824]                              


In [39]:
for indexes in list(ans_valid)[:100]:
    sen = ""
    for w_id in indexes[1:]:
        sen += id2word[str(int(w_id))]#+" "
    print(sen)

9年，我們的《告台灣同胞書》發表40週年紀念會上，我們貫徹十九大《告台灣同胞書》的重要講話，在十九大40週年紀念會的重要講話和支援的重要舉措<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>
道，2008年到對兩岸交流的交流合作對兩岸人員往來越多的限制、限制了兩岸交流；對兩岸的人員往來對於社會文化、社會發展的交流與合作的局面<eos>了<eos>的<eos><eos><eos><eos><eos>的<eos><eos><eos><eos><eos><eos><eos><eos>的<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>
續進一步促進兩岸經濟文化交流合作、促進的經濟文化交流合作，為台灣同胞來大陸學習、工作、生活提供更多的便利，積極推動兩岸青年的交流合作、惠及台胞來大陸學習、工作的政策措施<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>
胞交流合作經受了經受著密切，近年來兩岸人員往來，給兩岸交流基地的規模持續增長了，在台舉辦<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eo

In [41]:
def fgim_attack(input_indexes, output_indexes, label, batch_size): #model=dis_model, origin_data=latent
    label = 1-label
    origin_data = model.encode(input_indexes) #get latent
    dis_criterion = nn.CrossEntropyLoss(size_average=True)
    
    step_output = { i:[] for i in range(batch_size)}
    for epsilon in [2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]:
#         print(epsilon)
        it = 0
        data = origin_data
        while True:
            data = data.clone().detach()  #(batch_size, seq_length, latent_size)
            data.requires_grad = True
            cls_output = classifier.forward(data)
            loss = dis_criterion(cls_output, label)
            
            classifier.zero_grad()
            loss.backward()
            data_grad = data.grad.data
            data = data - epsilon * data_grad

            it += 1
            epsilon = epsilon * 0.9
            _, manip_output = model.decode(output_indexes, data) #manip_output=(batch size, hid size)
            for b_i, b in enumerate(manip_output.tolist()):
                step_output[b_i].append(b)
            if it >= 5:
                break
    for b in list(step_output.values()):
        for indexes in b:
            sen = ""
            for w_id in indexes[1:]:
                sen += id2word[str(int(w_id))]#+" "
            print(sen)
    return list(step_output.values())

In [42]:
def run_fgim(batch_size):
    #prepare dataloader
    mode='valid'
    dataset = trainset
    shuffle = False
    teacher_forcing = 0
    model.eval() #set the module in evaluation mode, only affect dropout, batchnorm
    
    batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4, \
                            collate_fn=lambda x:collate_fn(x))
    #train model
    t = tqdm(enumerate(batched_dataset), total=len(dataset)/batch_size)
    total_loss = 0
    cls_total_loss = 0
    ans = []
    gt = []
    for i, batch in t: #run batch
        batch_text = batch[0].to(device)
        batch_label = batch[1].to(device)
        batch_ans = fgim_attack(batch_text, batch_text, batch_label, batch_size)
        ans.extend(batch_ans)
        gt.extend(batch_label.tolist())
    return ans, gt

In [ ]:
ans, gt = run_fgim(batch_size=128):